# Common functions (Jupyter Notebooks and Self Driving Cars Udacity Nanodegree)

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', 
# for example, call as plt.imshow(gray, cmap='gray')

## Helper Functions

In [ ]:
import os
filenames = os.listdir("test_images/")

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)



In [ ]:
## Videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
Display Side by Side images

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(grad_binary, cmap='gray')
ax2.set_title('Thresholded Gradient', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
## Camera Calibration

In [ ]:
# Now based on al objpoints + imgpoints, will compute camera calibration matrix 
# and distortion coefficients (mtx and dist)
#ret, mtx, dist, rvecs, tvers = cv2.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)
ret, mtx, dist, rvecs, tvers = cv2.calibrateCamera(objpoints,imgpoints,(xSize,ySize),None,None)

In [ ]:
def cameraCalibration() :
    # images for camera calibration are stored in the folder called `camera_cal`.  
    # need to set your chessboard size to 9x6 for the project instead of 8x6 as in the lesson.

    # multiple pictures of the chessboard against a flat surface, 
    # then we’ll be able to detect any distortions by  looking at the difference between apparent 
    # size and the shape of squares in these images, and the size and shape that they actually are.
    #    • Then we’ll use that information to calibrate our camera.
    #    • Create a transform that maps distorted point to undistorded points.
    #    • And finally, undistort any images.
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.


    # Make a list of calibration images
    #images = glob.glob('camera_cal/calibration*.jpg')
    filenames = os.listdir(cameraCalFolder)

    # Step through the list and search for chessboard corners
    for fname in filenames :
        img = mpimg.imread(cameraCalFolder + '/' + fname)

        ret, corners, objpoints, imgpoints = findChessboardCorners(img, objpoints, imgpoints)

    # Now based on al objpoints + imgpoints, will compute camera calibration matrix 
    # and distortion coefficients (mtx and dist)
    ret, mtx, dist, rvecs, tvers = cv2.calibrateCamera(objpoints,imgpoints,(xSize,ySize),None,None)  
    
    return mtx, dist
    # end cameraCalibration()


In [ ]:
# Function that takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    ret, mtx, dist, rvecs, tvers = cv2.calibrateCamera(objpoints,imgpoints,img.shape[::-1],None,None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def display2ImagesSideBySide(img1,txt1,img2,txt2) :
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title(txt1, fontsize=50)
    ax2.imshow(img2)
    ax2.set_title(txt2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

def saveImageOutputAndShowVsImageInput(saveFolder, filenames_in, fname, img_in, legend_in,
                                       img_out, legend_out,saveGray=False) :
    # save and show output images -- for Writeup illustrations
    if saveImages :
        if not saveGray :
            mpimg.imsave( saveFolder + '/' + fname,img_out)
        else : 
            mpimg.imsave( saveFolder + '/' + fname,img_out,cmap='gray')
    # display exemple for first chessboard image
    if fname == filenames_in[0] :
        display2ImagesSideBySide(img_in,legend_in,img_out,legend_out)
        
def saveImageOutputAndShow(saveFolder, filenames_in, fname, img_out,saveGray=False) :
    # save and show output images -- for Writeup illustrations
    if saveImages :
        if not saveGray :
            mpimg.imsave( saveFolder + '/' + fname,img_out)
        else : 
            mpimg.imsave( saveFolder + '/' + fname,img_out,cmap='gray')
    # display exemple for first chessboard image
    if fname == filenames_in[0] :
        plt.imshow(img_out)
    

def getSizeImages() : 
    # Make a list of Thresholded Binary Images
    filenames = os.listdir(testImageFolder)
    img = mpimg.imread(testImageFolder + '/' + filenames[0])
    return img.shape[1], img.shape[0]


# keep track of things like where your last several detections of the lane lines 
# were and what the curvature was, so you can properly treat new detections. 
# To do this, it's useful to define a Line() class to keep track of all the 
# interesting parameters you measure from frame to frame.
# Define a class to receive the characteristics of each line detection

class Line():
    def __init__(self):
        self.reset()
    def reset(self):
        # set all members to their initial value
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [ ]:
def getPerspectiveTransformMatrix():
        
    #define 4 source points src = np.float32([[,],[,],[,],[,]])
    src=np.float32([[265,678],[1042,678],[582,460],[702,460]])
    #src=np.float32([[253,678],[1054,678],[608,441],[672,441]])
    #define 4 destination points src = np.float32([[,],[,],[,],[,]])
    dst=np.float32([[265,ySize-1],[1042,ySize-1],[265,0],[1042,0]])
    #dst=np.float32([[253,ySize-1],[1054,ySize-1],[253,0],[1054,0]])

    # use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    
    return M, Minv

# Those variables will be used later on as well --> extracted from loops
M, Minv = getPerspectiveTransformMatrix()

In [ ]:
## Chessboard corner detection

In [ ]:
# images for camera calibration are stored in the folder called `camera_cal`.  
# need to set your chessboard size to 9x6 for the project instead of 8x6 as in the lesson.

# multiple pictures of the chessboard against a flat surface, 
# then we’ll be able to detect any distortions by  looking at the difference between apparent 
# size and the shape of squares in these images, and the size and shape that they actually are.
#    • Then we’ll use that information to calibrate our camera.
#    • Create a transform that maps distorted point to undistorded points.
#    • And finally, undistort any images.

def findChessboardCorners(img, objpoints, imgpoints) :
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # convert image to gray scale, 
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
         
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
    return ret, corners, objpoints, imgpoints


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

In [ ]:
def warpImage(img):
    
    img_size = (xSize,ySize)
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    return warped


In [ ]:
def computePolynomial2nddegree(coeff,y) : 
    x = ((coeff[0]*(y**2)) + coeff[1]*y + coeff[2]).astype(int)
    return x 


In [ ]:
def addCurveDeviationOverlay(laneBoundaryImg, left_fit, right_fit, ploty,left_curverad, right_curverad, deviation) :
    
    curv_avg = int((left_curverad+right_curverad)/2)
    
    # print radius in overlay on boundaryLaneImage ...
    # Radius of curvature = xxxx(m)
    # Vehicule is x.xxm left/right of center
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    if deviation <= 0 :
        side = 'left'
    else : 
        side = 'right'
    txt1 = f'Radius of Curvature = {curv_avg}(m)'
    txt2 = f'Vehicle is {np.absolute(deviation):.2f}m {side} of center'
    
    # cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
    cv2.putText(laneBoundaryImg, txt1, (50,50), font, 2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(laneBoundaryImg, txt2, (50,100), font, 2, (255, 255, 255), 2, cv2.LINE_AA)

    return laneBoundaryImg

In [ ]:
# TensorFlow Lab

In [ ]:
# Problem 1 - Implement Min-Max scaling for grayscale image data
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

In [ ]:
def download(url, file):
    """
    Download file from <url>
    :param url: URL to file
    :param file: Local file path
    """
    if not os.path.isfile(file):
        print('Downloading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')

# Download the training and test dataset.
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_train.zip', 'notMNIST_train.zip')
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_test.zip', 'notMNIST_test.zip')

# Make sure the files aren't corrupted
assert hashlib.md5(open('notMNIST_train.zip', 'rb').read()).hexdigest() == 'c8673b3f28f489e9cdf3a3d74e2ac8fa',\
        'notMNIST_train.zip file is corrupted.  Remove the file and try again.'
assert hashlib.md5(open('notMNIST_test.zip', 'rb').read()).hexdigest() == '5d3c7e653e63471c88df796156a9dfa9',\
        'notMNIST_test.zip file is corrupted.  Remove the file and try again.'

In [ ]:
def uncompress_features_labels(file):
    """
    Uncompress features and labels from a zip file
    :param file: The zip file to extract the data from
    """
    features = []
    labels = []

    with ZipFile(file) as zipf:
        # Progress Bar
        filenames_pbar = tqdm(zipf.namelist(), unit='files')
        
        # Get features and labels from all files
        for filename in filenames_pbar:
            # Check if the file is a directory
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    # Load image data as 1 dimensional array
                    # We're using float32 to save on memory space
                    feature = np.array(image, dtype=np.float32).flatten()

                # Get the the letter from the filename.  This is the letter of the image.
                label = os.path.split(filename)[1][0]

                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

# Get the features and labels from the zip files
train_features, train_labels = uncompress_features_labels('notMNIST_train.zip')
test_features, test_labels = uncompress_features_labels('notMNIST_test.zip')

# Limit the amount of data to work with a docker container
docker_size_limit = 150000
train_features, train_labels = resample(train_features, train_labels, n_samples=docker_size_limit)

# Set flags for feature engineering.  This will prevent you from skipping an important step.
is_features_normal = False
is_labels_encod = False

# Wait until you see that all features and labels have been uncompressed.
print('All features and labels uncompressed.')

In [ ]:
# Problem 1 - Implement Min-Max scaling for grayscale image data
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    # TODO: Implement Min-Max scaling for grayscale image data
    """
    Min, Max = [0;255]   --> [0.1;0.9]
    [x*Min/(max - min) + b = 0.1 ; x*Max/(max - min) + b =0.9]
    --> b=0.1, x = 0.9-b --> x = 0.8
    newvalue = x*value/255 + 0.1 = 0.8*value/255 + 0.1
    """
    return np.add(np.multiply(0.8/255,image_data),0.1)

In [ ]:
# Save the data for easy access
pickle_file = 'notMNIST.pickle'
if not os.path.isfile(pickle_file):
    print('Saving data to pickle file...')
    try:
        with open('notMNIST.pickle', 'wb') as pfile:
            pickle.dump(
                {
                    'train_dataset': train_features,
                    'train_labels': train_labels,
                    'valid_dataset': valid_features,
                    'valid_labels': valid_labels,
                    'test_dataset': test_features,
                    'test_labels': test_labels,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

# Reload the data
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  train_features = pickle_data['train_dataset']
  train_labels = pickle_data['train_labels']
  valid_features = pickle_data['valid_dataset']
  valid_labels = pickle_data['valid_labels']
  test_features = pickle_data['test_dataset']
  test_labels = pickle_data['test_labels']
  del pickle_data  # Free up memory


In [ ]:
Pad images

In [ ]:
import numpy as np

# Pad images with 0s
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

In [ ]:
# Visualize Data

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

plt.figure(figsize=(1,1))
plt.imshow(image, cmap="gray")
print(y_train[index])

In [ ]:
# Shuffle data

In [ ]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
# Le Net

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
def printMeanAndVarianceDataSet(dataset,name):
    print()
    print(f'Few statistics about dataset {name} :\n###############################')
    print(f'- Values average/mean : {np.mean(dataset):.2f}')
    print(f'- Values average standard deviation : {np.mean(np.std(dataset,axis=0)):.2f}')
    print(f'- Mininum value : {np.amin(dataset):.2f}')
    print(f'- Maximum value : {np.amax(dataset):.2f}')

In [ ]:
# Extract table to match label indexes with German sign names :

In [ ]:
import csv

def read_signIndexNames() :
    # Read file 'signnames.csv', store in dictionary to map label indexes with sign titles
    # Format : ClassId,SignName
    sign_name_file = './signnames.csv'
    with open(sign_name_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        table = []
        for row in reader:
            #print(row['ClassId'], row['SignName'])
            table.append(row['SignName'])
    return table
        
sign_table = read_signIndexNames()
print(f'Exemple of label for image[0] : \"{sign_table[0]}\"')

In [ ]:
# Histogram of image labels y_train

In [ ]:
import matplotlib.pyplot as plt

# histogram on y_train
# Nb labels : n_classes

fig, ax = plt.subplots(figsize=(15,7))
# the histogram of the data
n, bins, patches = ax.hist(y_train,bins=n_classes,rwidth=0.8,align='left',
                    range=(0,n_classes))

ax.set_xlabel('Sign indexex')
ax.set_ylabel('nb of sign labels')
ax.set_title('Label Histogram')
ax.axes.grid(b=True,which='major',axis='y')
#ax.set_xticks(list(range(0,n_classes)), sign_table)
plt.xticks(list(range(0,n_classes)), sign_table,rotation=90)
# Tweak spacing to prevent clipping of ylabel
#fig.tight_layout()
plt.show()

In [ ]:
def displayImagesSideBySide(img_list) :
    nb_img = len(img_list)
    f, ax = plt.subplots(nrows=1, ncols=nb_img, figsize=(9, 9))
    f.tight_layout()
    for i in range(nb_img) :
        # choose cmap option if 3 color channels or if 1 (grayscale)
        img_cmap = 'gray' if (img_list[i].shape[2] == 1) else None 
        ax[i].imshow(img_list[i].squeeze(),img_cmap)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
def convertToGrayScale(images):
    images_gray = np.zeros(images.shape[:-1])
    for i in range(images.shape[0]): 
        images_gray[i] = cv2.cvtColor(images[i], cv2.COLOR_RGB2GRAY) 
    # add channel dimension at the end
    images_gray = np.expand_dims(images_gray, axis=3)
    return images_gray

displayImagesSideBySide(img_list=[original_img,X_train[index_random]])



In [ ]:
# Normalization for data 0 mean and equal variant¶

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.


def normalize(image_data): 
    # Shape (32, 32, 3) or (32,32) if converted to gray scale.
    return (np.divide(np.subtract(np.float32(image_data),np.float32(128)),128))

print(f'len(X_train) = {len(X_train)} len(y_train) = {len(y_train)} len(X_valid) = {len(X_valid)} len(y_valid) = {len(y_valid)} len(X_test) = {len(X_test)} len(y_test) = {len(y_test)}')
X_train = normalize(X_train)
X_valid = normalize(X_valid)
X_test = normalize(X_test)
print('info debug : ')
print(f'np.amax(X_train) = {np.amax(X_train)}') #[0][0])
print(f'np.amin(X_train) = {np.amin(X_train)}') #[0][0])
print(f'len(X_train) = {len(X_train)} len(y_train) = {len(y_train)} len(X_valid) = {len(X_valid)} len(y_valid) = {len(y_valid)} len(X_test) = {len(X_test)} len(y_test) = {len(y_test)}')

printMeanAndVarianceDataSet(X_train,'X_train after pre-processing')

In [ ]:
# Plotting X_train pixel values after normalization

In [ ]:
# plot_values(X_train,400000)
# Try to plot side by side X_train before and after normalization

def plot_values(dataset,nb=1):
    # N = 10000
    # dataset = X_train.flatten()
    dataset = dataset.flatten()
    N = min(nb,len(dataset))
    dataset = dataset[:N]
    x = list(range(0,N))
    plt.scatter(x, dataset, s=1)
    plt.show()
    
def plotDataSetBeforeAfter(data1,data2,nb=1) :
    data1 = data1.flatten()
    data2 = data2.flatten()
    N = min(nb,len(data1),len(data2))
    data1 = data1[:N]
    data2 = data2[:N]
    x = list(range(0,N))
    
    f, ax = plt.subplots(nrows=1, ncols=2,figsize=(10, 3))  # figsize=(9, 9)
    #f.tight_layout()
    ax[0].scatter(x, data1, s=1)
    ax[1].scatter(x, data2, s=1)
    ax[0].set_title('X_train pix values before normalization')
    ax[1].set_title('X_train pix values after normalization')
    #ax[0].show()
    #ax[1].show()
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plotDataSetBeforeAfter(X_train_original,X_train,nb=200000)

In [ ]:
# LeNet Traffic Sign Classifier

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x, in_channels):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
        
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, in_channels, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # Test DROPOUT to prevent overfitting.
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    
    # Test DROPOUT to prevent overfitting.
    fc2 = tf.nn.dropout(fc2, keep_prob)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    # Test DROPOUT to prevent overfitting.
    logits = tf.nn.dropout(logits, keep_prob)
    
    return logits

In [ ]:
# Load and Output the Images
- Problem : images (.png) were RGBA (32x32x4) instead of RGB (32x32x3).
- So I had to use PIL.image to read .png in RGBA and convert to RGB(32x32x3), using `convert('RGB')`, to make sure that images are in the format accepted by the LeNet pipeline above (32x32x3).


In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import matplotlib.image as mpimg
from os.path import isfile

import PIL.Image
    
web_image_folder = './webImages'

filenames = os.listdir(web_image_folder)
image_list = []
for fname in filenames : 
    if isfile(web_image_folder + '/' + fname) :
        image_list.append(np.array(
            PIL.Image.open(web_image_folder + '/' + fname).convert('RGB')))
displayImagesSideBySide(image_list)        
